Fine-tune FLAN-T5 for chat & dialogue summarization
https://www.philschmid.de/fine-tune-flan-t5

#### Load the dataset from huggingface and save to local disk for future use

In [ ]:
!pip install py7zr --quiet

In [ ]:
from datasets import load_dataset, load_from_disk

dataset_id = "samsum"
# Load dataset from the hub
dataset_hf = load_dataset(dataset_id)

print(f"Train dataset size: {len(dataset_hf['train'])}")
print(f"Test dataset size: {len(dataset_hf['test'])}")

# Train dataset size: 14732
# Test dataset size: 819

In [ ]:
# Save the dataset to local disk for future use
dataset_hf.save_to_disk('./dataset')

#### Load the dataset from local disk

In [ ]:
dataset = load_from_disk('./dataset')
print(f"Train dataset size: {len(dataset['train'])}")
print(f"Test dataset size: {len(dataset['test'])}")

In [ ]:
sample = dataset['test'][2]
print(f"dialogue: \n{sample['dialogue']}\n---------------")
print(f"summary: \n{sample['summary']}\n---------------")

In [ ]:
sample['dialogue']

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_id="google/flan-t5-small"

# Load tokenizer of FLAN-t5-base
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
max_source_length = 512
max_target_length = 128

In [ ]:
def preprocess_function(sample,padding="max_length"):
    # add prefix to the input for t5
    inputs = ["summarize: " + item for item in sample["dialogue"]]

    # tokenize inputs
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(text_target=sample["summary"], max_length=max_target_length, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=["dialogue", "summary", "id"])
print(f"Keys of tokenized dataset: {list(tokenized_dataset['train'].features)}")

In [ ]:
dataset

In [ ]:
#model_inputs = tokenizer("This is a test", max_length=max_source_length, padding="max_length", truncation=True)
#model_inputs

In [ ]:
tokenized_dataset

In [ ]:
str(tokenized_dataset['train'][2]['labels'])

In [ ]:
from transformers import AutoModelForSeq2SeqLM

# huggingface hub model id
model_id="google/flan-t5-small"

# load model from the hub
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

In [ ]:
from transformers import DataCollatorForSeq2Seq

# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)


In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# Hugging Face repository id
repository_id = f"{model_id.split('/')[1]}-{dataset_id}"

In [ ]:
# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=repository_id,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    predict_with_generate=True,
    fp16=False, # Overflows with fp16
    learning_rate=5e-5,
    num_train_epochs=5,
    # logging & evaluation strategies
    logging_dir=f"{repository_id}/logs",
    logging_strategy="steps",
    logging_steps=500,
    evaluation_strategy="steps",
    save_strategy="steps",
    save_total_limit=2,
    load_best_model_at_end=True,
    # metric_for_best_model="overall_f1"
)


In [ ]:
# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    #compute_metrics=compute_metrics,
)

In [ ]:
# Start training
trainer.train()


In [ ]:
!python ./scripts/train_flan_t5_sm_compatible.py --epochs 1 --training_dir "./dataset" --output_dir "./model" --train_batch_size 8